In [1]:
import torch
import os
from gan.models import Generator
from gan.config import TrainingConfig
from gan.utils import tensor_to_level_str,check_playable,check_level_similarity

config = TrainingConfig()
config.set_env()
device = torch.device("cuda")
model_save_path = "/root/mnt/GVGAI-GAN/gan/checkpoints/none-708"

generator = Generator(
    out_dim=config.input_shape[0],
    shapes=config.model_shapes,
    z_shape=(config.latent_size,),
    filters=config.generator_filters,
    is_self_attention=config.is_self_attention_g,
    is_conditional=config.is_conditional,
).to(device)

model_dict = torch.load(os.path.join(model_save_path, "models.tar"))
generator.load_state_dict(model_dict["generator"])




AttributeError: 'TrainingConfig' object has no attribute 'is_self_attention_g'

In [ ]:
def check_level_hamming(level1: str, level2: str):
    hit = 0
    for c1, c2 in zip(level1, level2):
        if c1 == "\n":
            continue
        if c1 != c2:
            hit += 1
    return hit


In [ ]:
latents_for_eval = torch.randn(
    300,
    config.latent_size,
).to(device)

output_levels = generator(latents_for_eval)

level_strs = tensor_to_level_str(config.env_name, output_levels)
playable_levels = []
playable_count = 0
for level_str in level_strs:
    if check_playable(level_str):
        playable_levels.append(level_str)

res_level= 0
n=0
for i in range(0, len(playable_levels)):
    for j in range(i + 1, len(playable_levels)):
        res_level += check_level_hamming(
            playable_levels[i], playable_levels[j])
        n+=1

print(len(playable_levels)/300)
print(res_level/(n)/(12*16))
print(24.34/(9*13))

0.18
0.11522027137200093
0.20803418803418802


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_good/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_good/",file),'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))

0.1607152342422061


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_large/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/generated_large/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))


0.12806500171009358


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/originals/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/originals/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))


0.2255208333333333


In [ ]:
import glob

files = glob.glob("/root/mnt/GVGAI-GAN/gan/data/level/zelda/random/*")

real_levels = []
for file in files:
    with open(os.path.join("/root/mnt/GVGAI-GAN/gan/data/level/zelda/random/", file), 'r') as f:
        s = f.read()
        real_levels.append(s)
hamming = 0
n = 0
for i in range(0, len(real_levels)):
    for j in range(i + 1, len(real_levels)):
        hamming += check_level_hamming(
            real_levels[i], real_levels[j])
        n += 1
print(hamming/(n)/(12*16))


0.24803135521885522


In [2]:
if config.cuda:
    device = torch.device(
        "cuda" if torch.cuda.is_available else "cpu")
    print("device : cuda")
else:
    device = torch.device("cpu")
    print("device : cpu")
generator = Generator(
    out_dim=config.input_shape[0],
    shapes=config.model_shapes,
    z_shape=(config.latent_size,),
    filters=config.generator_filters,
    use_conditional=config.use_conditional
).to(device)

model_path = os.path.join(
    "/root/mnt/pcg/GVGAI-GAN/gan/checkpoints/none-795", "models.tar")
load_model = torch.load(model_path)
generator.load_state_dict(load_model["generator"])

x = torch.randn(config.latent_size).to(device)

# es = cma.CMAEvolutionStrategy(config.latent_size*[0], 0.5)
# es.optimize(fitness)

def fitness(x: torch.Tensor):
    x = np.array(x)
    latent = torch.FloatTensor(x).view(1, -1, 1, 1)
    level = generator(x)
    level_str = tensor_to_level_str(level)
    playable = check_playable(level_str)
    eval = eval(level_str)
    return playable*300 + eval

def eval(level_str):
    wall = 0
    enemy = 0
    for s, i in enumerate(level_str):
        for c, j in enumerate(level_str):
            if c == 'w':
                wall += 1
            if c in ['1', '2', '3']:
                enemy += 1
    return wall + enemy


device : cuda


TypeError: __init__() got an unexpected keyword argument 'use_conditional'